In [1]:
import json
import csv
import os
import time
import psutil  
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import squarify

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']

In [2]:
"""
The original file is in JSON format, and we will convert it into CSV format and 
save it as "processed_sports_facilities_11.csv" for easier management in the future.

Since some of the naming in this data is ambiguous and fragmented, we will categorize 
facilities with the same function into groups to streamline subsequent management. 
For example, 'BA10', 'BA11', and 'BA12' all represent badminton courts, so we will 
classify them under the same category.
"""

'\nThe original file is in JSON format, and we will convert it into CSV format and \nsave it as "processed_sports_facilities_11.csv" for easier management in the future.\n\nSince some of the naming in this data is ambiguous and fragmented, we will categorize \nfacilities with the same function into groups to streamline subsequent management. \nFor example, \'BA10\', \'BA11\', and \'BA12\' all represent badminton courts, so we will \nclassify them under the same category.\n'

In [ ]:
def process_large_json_to_csv(input_file_path, output_csv_path, chunk_size=5000):

    
    def extract_nested_values(record):
        try:
            
            processed = {
                '_id': record.get('_id', {}).get('$oid', ''),
                'bookingID': int(record.get('bookingID', {}).get('$numberLong', 0)),
                'bookingDate': record.get('bookingDate', {}).get('$date', ''),
                'placeCode': record.get('placeCode', ''),
                'timeSlot': int(record.get('timeSlot', {}).get('$numberLong', 0)),
                'bookingStatus': record.get('bookingStatus', ''),
                'lastModifiedDate': record.get('lastModifiedDate', {}).get('$date', '')
            }
            return processed
        except Exception as e:
            print(f"Record processing error: {e}")
            return None
    
    def print_memory_usage():
        """Print current memory usage"""
        process = psutil.Process(os.getpid())
        mem = process.memory_info().rss / (1024 * 1024)  
        print(f"Memory usage: {mem:.2f} MB")
    
    
    start_time = time.time()
    total_records = 0
    processed_records = 0
    chunk_count = 0
    current_chunk = []
    
    try:
        with open(input_file_path, 'r', encoding='utf-8') as f:
            total_records = sum(1 for _ in f)
        print(f"Total records in file: {total_records}")
    except Exception as e:
        print(f"Cannot get total lines in file: {e}")
        total_records = 0
    
    print(f"Starting file processing: {input_file_path}")
    print(f"Chunk size: {chunk_size} records")
    print_memory_usage()
    
    try:
        
        with open(output_csv_path, 'w', encoding='utf-8', newline='') as csv_file:
            fieldnames = [
                '_id', 'bookingID', 'bookingDate', 'placeCode', 
                'timeSlot', 'bookingStatus', 'lastModifiedDate'
            ]
            csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
            csv_writer.writeheader()
            
            with open(input_file_path, 'r', encoding='utf-8') as json_file:
                line_number = 0
                
                for line in json_file:
                    line_number += 1
                    line = line.strip()
                    if not line:
                        continue
                    
                    # Display progress
                    if line_number % 10000 == 0:
                        elapsed = time.time() - start_time
                        speed = line_number / elapsed if elapsed > 0 else 0
                        print(f"Processed: {line_number}/{total_records} lines | "
                              f"Progress: {line_number/total_records*100:.1f}% | "
                              f"Speed: {speed:.0f} lines/second")
                        print_memory_usage()
                    
                    try:
                        # Parse JSON line
                        record = json.loads(line)
                        # Process nested values
                        processed_record = extract_nested_values(record)
                        
                        if processed_record:
                            current_chunk.append(processed_record)
                            processed_records += 1
                        
                        # Write to CSV when chunk size is reached
                        if len(current_chunk) >= chunk_size:
                            chunk_count += 1
                            csv_writer.writerows(current_chunk)
                            current_chunk = []
                    
                    except json.JSONDecodeError as e:
                        print(f"Line {line_number} JSON parsing error: {e}")
                        print(f"Problematic line content: {line[:200]}...")  # Show first 200 characters
                    except Exception as e:
                        print(f"Line {line_number} processing error: {e}")
            
            # Process the last incomplete chunk
            if current_chunk:
                csv_writer.writerows(current_chunk)
    
    except FileNotFoundError:
        print(f"Error: File not found {input_file_path}")
        return None
    except Exception as e:
        print(f"Error during processing: {e}")
        return None
    
    # Calculate total statistics
    total_time = time.time() - start_time
    print(f"\n✅ Processing completed!")
    print(f"Total processed records: {processed_records}")
    print(f"Total data chunks: {chunk_count}")
    print(f"Total time taken: {total_time:.2f} seconds")
    
    if total_time > 0:
        print(f"Average speed: {processed_records/total_time:.2f} records/second")
    
    return output_csv_path

# Usage example
if __name__ == "__main__":
    # Configure file paths
    input_json_file = "../data/sports_facilities_booking_2023.json"  # Replace with your JSON file path
    output_csv_file = "../data/processed_sports_facilities_11.csv"      # Output CSV file
    
    # Install required libraries (if not installed)
    try:
        import psutil
    except ImportError:
        print("Installing psutil for memory monitoring...")
        import subprocess
        subprocess.run(["pip", "install", "psutil"])
        import psutil
    
    # Execute conversion
    processed_csv = process_large_json_to_csv(
        input_file_path=input_json_file,
        output_csv_path=output_csv_file,
        chunk_size=5000  # Smaller chunk size is safer
    )
    
    if processed_csv:
        print(f"CSV file successfully created: {processed_csv}")
        print(f"File size: {os.path.getsize(processed_csv)/(1024 * 1024):.2f} MB")

Total records in file: 611253
Starting file processing: sports_facilities_booking_2023.json
Chunk size: 5000 records
Memory usage: 171.83 MB
Processed: 10000/611253 lines | Progress: 1.6% | Speed: 83812 lines/second
Memory usage: 175.55 MB
Processed: 20000/611253 lines | Progress: 3.3% | Speed: 132480 lines/second
Memory usage: 175.55 MB
Processed: 30000/611253 lines | Progress: 4.9% | Speed: 163415 lines/second
Memory usage: 175.55 MB
Processed: 40000/611253 lines | Progress: 6.5% | Speed: 184806 lines/second
Memory usage: 175.55 MB
Processed: 50000/611253 lines | Progress: 8.2% | Speed: 200783 lines/second
Memory usage: 175.55 MB
Processed: 60000/611253 lines | Progress: 9.8% | Speed: 213934 lines/second
Memory usage: 175.55 MB
Processed: 70000/611253 lines | Progress: 11.5% | Speed: 223569 lines/second
Memory usage: 175.55 MB
Processed: 80000/611253 lines | Progress: 13.1% | Speed: 212095 lines/second
Memory usage: 175.56 MB
Processed: 90000/611253 lines | Progress: 14.7% | Speed: 2